### Realizado por: 
#### Desiderio Nevado López
#### Laura Martín Olinero 
#### Daniel Quesada Pérez de San Roman

# Apartado 2 Consulta 1

### En primer lugar, creamos el consumer y producer.
### Arrancamos docker compose

### Utilizando la API para programación con Spark Structured Streaming, y configurando un intervalo de actualización de micro-batches (trigger respecto del ProcessingTime) de 3 segundos1 , se pide resolver las siguientes consultas:

### 1. Calcula la media de los valores de temperatura, humedad relativa y concentración de CO2 para cada micro-batch, así como la media de dichos valores desde el arranque de la aplicación.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, window, col, mean, stddev
from pyspark.sql.types import StringType, StructType, StructField, TimestampType, LongType, DoubleType
import time

# Iniciamos sesion de Spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Consulta1_Apartado2') \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()

# Establecemos un nuevo nivel de log para evitar warning al hacer login
spark.sparkContext.setLogLevel('error')

in_path = '/home/desi/Descargas/iot_telemtry_data.csv'
out_path = '/home/desi/Descargas/Consulta1Apartado2'
kafka_host = 'localhost'
kafka_port = 9092
kafka_topic = 'iot'

23/07/23 21:59:07 WARN Utils: Your hostname, jose-Blade-15-Advanced-Model-Mid-2021-RZ09-0409 resolves to a loopback address: 127.0.1.1; using 192.168.1.48 instead (on interface wlp91s0)
23/07/23 21:59:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/jose/Documentos/pyspark/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jose/.ivy2/cache
The jars for the packages stored in: /home/jose/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e81be1bb-e132-4f0b-9df5-31d5bdadb76f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 266ms :: artifacts dl 14ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central in 

23/07/23 21:59:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# Para leer los datos

stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "iot") \
    .load() \
    .selectExpr("CAST(value AS STRING)")

In [3]:
from pyspark.sql.functions import split, expr, col

stream = stream.withColumn('value', expr("regexp_replace(value, '\"','')"))

df = (
    stream
    .select(
        split(col('value'), ',')[0].cast('float').alias('ts'),
        split(col('value'), ',')[1].alias('device'),
        split(col('value'), ',')[2].cast('float').alias('co'),
        split(col('value'), ',')[3].cast('float').alias('humidity'),
        split(col('value'), ',')[4].cast('boolean').alias('light'),
        split(col('value'), ',')[5].cast('float').alias('lpg'),
        split(col('value'), ',')[6].cast('boolean').alias('motion'),
        split(col('value'), ',')[7].cast('float').alias('smoke'),
        split(col('value'), ',')[8].cast('float').alias('temp'),
    )
)

In [4]:
from pyspark.sql.functions import from_unixtime, col, avg, window
from pyspark.sql.types import TimestampType

# Transform the 'ts' column from float (unix timestamp) to timestamp
df = df.withColumn('ts', from_unixtime(col('ts')).cast(TimestampType()))

# Setting watermark of 3 seconds to handle late data
avg_temp_hum_co = df.withWatermark('ts', '3 seconds') \
    .groupBy(window('ts', '3 seconds')) \
    .agg(avg('temp').alias('avg_temp_total'),
         avg('humidity').alias('avg_humidity_total'),
         avg('co').alias('avg_co_total')) \
    .select('window.*', 'avg_temp_total', 'avg_humidity_total', 'avg_co_total')

query = avg_temp_hum_co.writeStream \
    .outputMode('update') \
    .format('console') \
    .option('truncate', 'false') \
    .trigger(processingTime='3 seconds') \
    .start()

query.awaitTermination(timeout=30)

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+---+--------------+------------------+------------+
|start|end|avg_temp_total|avg_humidity_total|avg_co_total|
+-----+---+--------------+------------------+------------+
+-----+---+--------------+------------------+------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+-------------------+------------------+------------------+--------------------+
|start              |end                |avg_temp_total    |avg_humidity_total|avg_co_total        |
+-------------------+-------------------+------------------+------------------+--------------------+
|2020-07-12 03:23:12|2020-07-12 03:23:15|23.566667238871258|59.81666819254557 |0.004263860445159177|
+-------------------+-------------------+------------------+------------------+--------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+-------------------+-------------------+------------------+------------------+-------------------+
|start              |end                |avg_temp_total    |avg_humidity_total|avg_co_total       |
+-------------------+-------------------+------------------+------------------+-------------------+
|2020-07-12 03:23:12|2020-07-12 03:23:15|23.330000495910646|61.510000991821286|0.00420660621020943|
+-------------------+-------------------+------------------+------------------+-------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+-------------------+-------------------+------------------+------------------+--------------------+
|start              |end                |avg_temp_total    |avg_humidity_total|avg_co_total        |
+-------------------+-------------------+------------------+------------------+--------------------+
|2020-07-12 03:23:12|2020-07-12 03:23:15|23.235714776175364|62.23571532113211 |0.004186467849649489|
+-------------------+-------------------+------------------+------------------+--------------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+-------------------+-------------------+------------------+------------------+--------------------+
|start              |end                |avg_temp_total    |avg_humidity_total|avg_co_total        |
+-------------------+-------------------+------------------+------------------+--------------------+
|2020-07-12 03:23:12|2020-07-12 03:23:15|23.215789895308646|64.71579019646896 |0.004038767251921327|
+-------------------+-------------------+------------------+------------------+--------------------+



...

-------------------------------------------
Batch: 6
-------------------------------------------
+-------------------+-------------------+------------------+------------------+--------------------+
|start              |end                |avg_temp_total    |avg_humidity_total|avg_co_total        |
+-------------------+-------------------+------------------+------------------+--------------------+
|2020-07-12 03:23:12|2020-07-12 03:23:15|23.028571810041154|64.97142900739398 |0.004007764193894607|
+-------------------+-------------------+------------------+------------------+--------------------+



-------------------------------------------
Batch: 7
-------------------------------------------
+-------------------+-------------------+------------------+------------------+--------------------+
|start              |end                |avg_temp_total    |avg_humidity_total|avg_co_total        |
+-------------------+-------------------+------------------+------------------+--------------------+
|2020-07-12 03:23:12|2020-07-12 03:23:15|23.009091290560637|64.4606064305161  |0.004052188303886038|
+-------------------+-------------------+------------------+------------------+--------------------+



-------------------------------------------
Batch: 8
-------------------------------------------
+-------------------+-------------------+------------------+------------------+--------------------+
|start              |end                |avg_temp_total    |avg_humidity_total|avg_co_total        |
+-------------------+-------------------+------------------+------------------+--------------------+
|2020-07-12 03:23:12|2020-07-12 03:23:15|23.100000381469727|64.76842137386924 |0.004063843191895438|
+-------------------+-------------------+------------------+------------------+--------------------+

-------------------------------------------


False

Batch: 9
-------------------------------------------


In [6]:
spark.stop()